# Developing Word Embeddings

Rather than use pre-trained embeddings (as we did in the sentence similarity baseline_deep_dive [notebook](../sentence_similarity/baseline_deep_dive.ipynb)), we can train word embeddings using our own dataset. In this notebook, we demonstrate the training process for producing word embeddings using the word2vec, GloVe, and fastText models. We'll utilize the STS Benchmark dataset for this task. 

# Table of Contents
* [Data Loading and Preprocessing](#Load-and-Preprocess-Data)
* [Word2Vec](#Word2Vec)
* [fastText](#fastText)
* [GloVe](#GloVe)
* [Concluding Remarks](#Concluding-Remarks)

In [8]:
import gensim
import sys
# Set the environment path
sys.path.append("../../") 
import os
import numpy as np
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)
from utils_nlp.dataset import stsbenchmark
from utils_nlp.common.timer import Timer
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText

In [9]:
# Set the path for where your datasets are located
BASE_DATA_PATH = "../../data" 
# Location to save embeddings
SAVE_FILES_PATH = BASE_DATA_PATH + "/trained_word_embeddings/"

In [10]:
if not os.path.exists(SAVE_FILES_PATH):
    os.makedirs(SAVE_FILES_PATH)

## Load and Preprocess Data

In [11]:
# Produce a pandas dataframe for the training set
train_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")

# Clean the sts dataset
sts_train = stsbenchmark.clean_sts(train_raw)

100%|██████████████████████████████████████████████████████████████████████████████████| 401/401 [00:01<00:00, 309KB/s]


Data downloaded to ../../data\raw\stsbenchmark


In [12]:
sts_train.head(5)

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


In [13]:
# Check the size of our dataframe
sts_train.shape

(5749, 3)

#### Training set preprocessing

In [14]:
# Convert all text to lowercase
df_low = to_lowercase(sts_train)  
# Tokenize text
sts_tokenize = to_spacy_tokens(df_low) 
# Tokenize with removal of stopwords
sts_train_stop = rm_spacy_stopwords(sts_tokenize) 

In [15]:
# Append together the two sentence columns to get a list of all tokenized sentences.
all_sentences =  sts_train_stop[["sentence1_tokens_rm_stopwords", "sentence2_tokens_rm_stopwords"]]
# Flatten two columns into one list and remove all sentences that are size 0 after tokenization and stop word removal.
sentences = [i for i in all_sentences.values.flatten().tolist() if len(i) > 0]

In [16]:
len(sentences)

11498

In [17]:
sentence_lengths = [len(i) for i in sentences]
print("Minimum sentence length is {} tokens".format(min(sentence_lengths)))
print("Maximum sentence length is {} tokens".format(max(sentence_lengths)))
print("Median sentence length is {} tokens".format(np.median(sentence_lengths)))

Minimum sentence length is 1 tokens
Maximum sentence length is 43 tokens
Median sentence length is 6.0 tokens


In [18]:
sentences[:10]

[['plane', 'taking', '.'],
 ['air', 'plane', 'taking', '.'],
 ['man', 'playing', 'large', 'flute', '.'],
 ['man', 'playing', 'flute', '.'],
 ['man', 'spreading', 'shreded', 'cheese', 'pizza', '.'],
 ['man', 'spreading', 'shredded', 'cheese', 'uncooked', 'pizza', '.'],
 ['men', 'playing', 'chess', '.'],
 ['men', 'playing', 'chess', '.'],
 ['man', 'playing', 'cello', '.'],
 ['man', 'seated', 'playing', 'cello', '.']]

## Word2Vec

Word2vec is a predictive model for learning word embeddings from text (see [original research paper](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)). Word embeddings are learned such that words that share common contexts in the corpus will be close together in the vector space. There are two different model architectures that can be used to produce word2vec embeddings: continuous bag-of-words (CBOW) or continuous skip-gram. The former uses a window of surrounding words (the "context") to predict the current word and the latter uses the current word to predict the surrounding context words. See this [tutorial](https://www.guru99.com/word-embedding-word2vec.html#3) on word2vec for more detailed background on the model.

The gensim Word2Vec model has many different parameters (see [here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between the word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm; 1 for skip-gram and 0 for CBOW (defaults to 0)

In [19]:
# Set up a Timer to see how long the model takes to train
t = Timer()

In [20]:
t.start()

# Train the Word2vec model
word2vec_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=3, sg=0)

t.stop()

In [21]:
print("Time elapsed: {}".format(t))

Time elapsed: 0.4556


Now that the model is trained we can:

1. Query for the word embeddings of a given word. 
2. Inspect the model vocabulary
3. Save the word embeddings

In [22]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", word2vec_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words.
print("\nFirst 30 vocabulary words:", list(word2vec_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format.
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=True)  # binary format
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=False)  # ASCII format

Embedding for apple: [ 0.05805362  0.06101197 -0.04139881  0.02149955 -0.09089632  0.08171839
  0.10880544  0.04739253 -0.18464622  0.18185261 -0.0273802   0.23335838
  0.02462817  0.19001065  0.042492   -0.03106086  0.13986434 -0.08186961
 -0.04803263 -0.03560257 -0.01290459 -0.05349363 -0.01384514 -0.19388926
 -0.07060098  0.06136238 -0.08374732 -0.07936234  0.14275725 -0.17378892
 -0.07579862  0.1358681   0.03124874  0.07999087 -0.10487169  0.03901242
 -0.03545398  0.1413099   0.06107847 -0.06615571  0.03585797 -0.1804256
  0.23718679  0.0819917  -0.17114222  0.06501587 -0.03194249 -0.05697308
 -0.16496892 -0.02637602  0.01153994 -0.10465483  0.16883366  0.03583959
 -0.05584354  0.11883577 -0.01215279 -0.2250833  -0.07159518  0.08646166
  0.00850767  0.07679912 -0.13213757 -0.08736049 -0.09475534 -0.03855689
  0.01396248 -0.02864163  0.00354996 -0.01462657 -0.08833787 -0.11314301
 -0.04131266 -0.09071928 -0.03713143  0.1178434  -0.12651944 -0.11256607
  0.13031591 -0.15850762  0.113

C:\Users\cocochra\AppData\Local\Continuum\anaconda3\envs\nlp_gpu\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## fastText

fastText is an unsupervised algorithm created by Facebook Research for efficiently learning word embeddings (see [original research paper](https://arxiv.org/pdf/1607.04606.pdf)). fastText is significantly different than word2vec or GloVe in that these two algorithms treat each word as the smallest possible unit to find an embedding for. Conversely, fastText assumes that words are formed by an n-gram of characters (i.e. 2-grams of the word "language" would be {la, an, ng, gu, ua, ag, ge}). The embedding for a word is then composed of the sum of these character n-grams. This has advantages when finding word embeddings for rare words and words not present in the dictionary, as these words can still be broken down into character n-grams. Typically, for smaller datasets, fastText performs better than word2vec or GloVe. See this [tutorial](https://fasttext.cc/docs/en/unsupervised-tutorial.html) on fastText for more detail.

The gensim fastText model has many different parameters (see [here](https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between the word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm- 1 for skip-gram and 0 for CBOW (defaults to 0)
- iter: number of epochs (defaults to 5)


In [23]:
# Set up a Timer to see how long the model takes to train
t = Timer()

In [24]:
t.start()

# Train the FastText model
fastText_model = FastText(size=100, window=5, min_count=5, sentences=sentences, iter=5)

t.stop()

In [25]:
print("Time elapsed: {}".format(t))

Time elapsed: 10.3698


We can utilize the same attributes as we saw above for word2vec due to them both originating from the gensim package

In [26]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", fastText_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words.
print("\nFirst 30 vocabulary words:", list(fastText_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format.
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=True)  # binary format
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=False)  # ASCII format

Embedding for apple: [-0.18175453 -0.14863092  0.01440668  0.41852772  0.4886491  -0.24110396
 -0.26591563 -0.42659786 -0.04840926 -0.05654079  0.26051033 -0.02733019
  0.00937179 -0.07287153  0.21057971  0.21508346  0.06344912  0.10872953
 -0.10214202 -0.54538804 -0.15845574 -0.05536952 -0.04718296 -0.46515992
 -0.12252445 -0.09347973  0.11549287  0.14775406 -0.4141621   0.24835227
  0.08907127 -0.00180367 -0.02042806  0.13677692  0.19265138  0.1525672
  0.05339279 -0.18745865 -0.38480887 -0.26928213  0.2699537   0.38778877
  0.28482276 -0.17511593  0.11898511 -0.06478633 -0.39813048  0.30248052
  0.03833921  0.08309021 -0.06976178 -0.15951832 -0.6560336  -0.4534666
 -0.18082033  0.09569218  0.10938869 -0.3292928  -0.4216524   0.24858503
 -0.35272446 -0.30754313  0.06224228  0.23139575 -0.11154156  0.03544799
 -0.09699723  0.13625555  0.3257419  -0.09298395  0.3291442  -0.03776973
 -0.17104091 -0.19018205  0.13310616  0.22434781 -0.00192542 -0.22643566
 -0.02940017 -0.3396929   0.0958

## GloVe

GloVe is an unsupervised algorithm for obtaining word embeddings created by the Stanford NLP group (see [original research paper](https://nlp.stanford.edu/pubs/glove.pdf)). Training occurs on word-word co-occurrence statistics with the objective of learning word embeddings such that the dot product of two words' embeddings is equal to the words' probability of co-occurrence. See this [tutorial](https://nlp.stanford.edu/projects/glove/) on GloVe for more detailed background on the model. 

Gensim doesn't have an implementation of the GloVe model and the other python packages that implement GloVe are unstable, so we suggest getting the code directly from the Stanford NLP [repo](https://github.com/stanfordnlp/GloVe). Run the following commands to clone the repo and then make. Clone the repo in the same location as this notebook! Otherwise, the paths below will need to be modified.  

    git clone http://github.com/stanfordnlp/glove    
    cd glove && make  

### Train GloVe vectors

Training GloVe embeddings requires some data prep and then 4 steps (also documented in the original Stanford NLP repo [here](https://github.com/stanfordnlp/GloVe/tree/master/src)).

**Step 0: Prepare Data**
   
In order to train our GloVe vectors, we first need to save our corpus as a text file with all words separated by 1+ spaces or tabs. Each document/sentence is separated by a new line character.

In [27]:
# Save our corpus as tokens delimited by spaces with new line characters in between sentences.
with open(BASE_DATA_PATH+'/clean/stsbenchmark/training-corpus-cleaned.txt', 'w', encoding='utf8') as file:
    for sent in sentences:
        file.write(" ".join(sent) + "\n")

In [28]:
# Set up a Timer to see how long the model takes to train
t = Timer()
t.start()

**Step 1: Build Vocabulary**

Run the vocab_count executable. There are 3 optional parameters:
1. min-count: lower limit on how many times a word must appear in dataset. Otherwise the word is discarded from our vocabulary.
2. max-vocab: upper bound on the number of vocabulary words to keep
3. verbose: 0, 1, or 2 (default)

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the vocabulary to 

In [29]:
!"glove/build/vocab_count" -min-count 5 -verbose 2 <"../../data/clean/stsbenchmark/training-corpus-cleaned.txt"> "../../data/trained_word_embeddings/vocab.txt"

BUILDING VOCABULARY
Processed 0 tokens.Processed 85334 tokens.
Counted 11716 unique words.
Truncating vocabulary at min count 5.
Using vocabulary of size 2943.



**Step 2: Construct Word Co-occurrence Statistics**

Run the cooccur executable. There are many optional parameters, but we list the top ones here:
1. symmetric: 0 for only looking at left context, 1 (default) for looking at both left and right context
2. window-size: number of context words to use (default 15)
3. verbose: 0, 1, or 2 (default)
4. vocab-file: path/name of the vocabulary file created in Step 1
5. memory: soft limit for memory consumption, default 4
6. max-product: limit the size of dense co-occurrence array by specifying the max product (integer) of the frequency counts of the two co-occurring words

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the co-occurrences to

In [30]:
!"glove/build/cooccur" -memory 4 -vocab-file "../../data/trained_word_embeddings/vocab.txt" -verbose 2 -window-size 15 <"../../data/clean/stsbenchmark/training-corpus-cleaned.txt"> "../../data/trained_word_embeddings/cooccurrence.bin"

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "../../data/trained_word_embeddings/vocab.txt"...loaded 2943 words.
Building lookup table...table contains 8661250 elements.
Processing token: 0Processed 85334 tokens.
Writing cooccurrences to disk......2 files in total.
Merging cooccurrence files: processed 0 lines.0 lines.100000 lines.Merging cooccurrence files: processed 188154 lines.



**Step 3: Shuffle the Co-occurrences**

Run the shuffle executable. The parameters are as follows:
1. verbose: 0, 1, or 2 (default)
2. memory: soft limit for memory consumption, default 4
3. array-size: limit to the length of the buffer which stores chunks of data to shuffle before writing to disk

Then provide the path to the co-occurrence file we created in Step 2 followed by a file path that we'll save the shuffled co-occurrences to

In [31]:
!"glove/build/shuffle" -memory 4 -verbose 2 <"../../data/trained_word_embeddings/cooccurrence.bin"> "../../data/trained_word_embeddings/cooccurrence.shuf.bin"

SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 188154 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.188154 lines.Merging temp files: processed 188154 lines.



**Step 4: Train GloVe model**

Run the glove executable. There are many parameter options, but the top ones are listed below:
1. verbose: 0, 1, or 2 (default)
2. vector-size: dimension of word embeddings (50 is default)
3. threads: number threads, default 8
4. iter: number of iterations, default 25
5. eta: learning rate, default 0.05
6. binary: whether to save binary format (0: text = default, 1: binary, 2: both)
7. x-max: cutoff for weighting function, default is 100
8. vocab-file: file containing vocabulary as produced in Step 1
9. save-file: filename to save vectors to 
10. input-file: filename with co-occurrences as returned from Step 3

In [32]:
!"glove/build/glove" -save-file "../../data/trained_word_embeddings/GloVe_vectors" -threads 8 -input-file \
"../../data/trained_word_embeddings/cooccurrence.shuf.bin" -x-max 10 -iter 15 -vector-size 50 -binary 2 \
-vocab-file "../../data/trained_word_embeddings/vocab.txt" -verbose 2

TRAINING MODEL
Read 188154 lines.
Initializing parameters...done.
vector size: 50
vocab size: 2943
x_max: 10.000000
alpha: 0.750000
06/26/19 - 09:24.10AM, iter: 001, cost: 0.078565
06/26/19 - 09:24.10AM, iter: 002, cost: 0.072320
06/26/19 - 09:24.10AM, iter: 003, cost: 0.070274
06/26/19 - 09:24.10AM, iter: 004, cost: 0.067244
06/26/19 - 09:24.10AM, iter: 005, cost: 0.063690
06/26/19 - 09:24.10AM, iter: 006, cost: 0.060640
06/26/19 - 09:24.10AM, iter: 007, cost: 0.058201
06/26/19 - 09:24.10AM, iter: 008, cost: 0.056211
06/26/19 - 09:24.10AM, iter: 009, cost: 0.054148
06/26/19 - 09:24.10AM, iter: 010, cost: 0.051913
06/26/19 - 09:24.10AM, iter: 011, cost: 0.049649
06/26/19 - 09:24.10AM, iter: 012, cost: 0.047426
06/26/19 - 09:24.10AM, iter: 013, cost: 0.045255
06/26/19 - 09:24.10AM, iter: 014, cost: 0.043138
06/26/19 - 09:24.10AM, iter: 015, cost: 0.041108


In [33]:
t.stop()

In [34]:
print("Time elapsed: {}".format(t))

Time elapsed: 25.0459


### Inspect Word Vectors

Like we did above for the word2vec and fastText models, let's now inspect our word embeddings

In [35]:
#load in the saved word vectors.
glove_wv = {}
with open("../../data/trained_word_embeddings/GloVe_vectors.txt", encoding='utf-8') as f:
    for line in f:
        split_line = line.split(" ")
        glove_wv[split_line[0]] = [float(i) for i in split_line[1:]]

In [36]:
# 1. Let's see the word embedding for "apple" by passing in "apple" as the key.
print("Embedding for apple:", glove_wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words.
print("\nFirst 30 vocabulary words:", list(glove_wv.keys())[:20])

Embedding for apple: [0.062942, -0.097984, 0.037373, 0.111635, 0.086733, -0.071781, 0.043611, -0.01458, -0.012725, 0.076614, -0.13072, 0.129127, -0.00262, 0.015669, 0.06114, -0.044421, 0.004353, -0.066637, 0.049023, -0.00885, 0.138072, 0.165017, 0.047256, 0.122998, -0.247253, 0.01951, 0.007255, 0.070611, -0.130033, -0.05971, 0.056946, -0.085183, -0.118371, 0.033433, -0.035763, 0.021646, -0.005461, -0.03758, -0.048107, -0.075025, 0.012993, -0.07799, -0.030288, -0.137319, 0.121737, 0.054742, -0.013201, 0.055261, -0.146741, -0.041641]

First 30 vocabulary words: ['.', ',', 'man', '-', '"', 'woman', "'", 'said', 'dog', 'playing', ':', 'white', 'black', '$', 'killed', 'percent', 'new', 'syria', 'people', 'china']


# Concluding Remarks

In this notebook we have shown how to train word2vec, GloVe, and fastText word embeddings on the STS Benchmark dataset. We also inspected how long each model took to train on our dataset: word2vec took 0.39 seconds, GloVe took 8.16 seconds, and fastText took 10.41 seconds.

FastText is typically regarded as the best baseline for word embeddings (see [blog](https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a)) and is a good place to start when generating word embeddings. Now that we generated word embeddings on our dataset, we could also repeat the baseline_deep_dive notebook using these embeddings (versus the pre-trained ones from the internet). 